<a href="https://colab.research.google.com/github/yeseul106/UROP_Software-Fault-Prediction/blob/main/UROP_IQR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [49]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
# 원본 csv 파일 읽어오기
JM1_original = pd.read_csv('/content/drive/MyDrive/UROP/UROP dataset/JM1_proff(PROMISE).csv', encoding='cp949')
print(JM1_original.shape)
JM1_original.head(5)

(10885, 22)


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,1.30,1.30,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1,1,1,1,1,1,1,1,1,True
2,72.0,7.0,1.0,6.0,198.0,1134.13,0.05,20.31,55.85,23029.10,0.38,1279.39,51,10,8,1,17,36,112,86,13,True
3,190.0,3.0,1.0,3.0,600.0,4348.76,0.06,17.06,254.87,74202.67,1.45,4122.37,129,29,28,2,17,135,329,271,5,True
4,37.0,4.0,1.0,4.0,126.0,599.12,0.06,17.19,34.86,10297.30,0.20,572.07,28,1,6,0,11,16,76,50,7,True


In [51]:
print(JM1_original.loc[JM1_original['uniq_Op'] == '?'])

       loc  v(g)  ev(g)  iv(g)  ...  total_Op  total_Opnd  branchCount  defects
143   49.0   4.0    3.0    3.0  ...         ?           ?            ?     True
358   27.0   5.0    3.0    3.0  ...         ?           ?            ?     True
1598  76.0  18.0   11.0   11.0  ...         ?           ?            ?     True
4214   6.0   1.0    1.0    1.0  ...         ?           ?            ?    False
8279   9.0  13.0   10.0    4.0  ...         ?           ?            ?    False

[5 rows x 22 columns]


In [52]:
# 이상 값이 있는 행 삭제
JM1_original = JM1_original.drop([143,358,1598,4214,8279], axis=0)

In [53]:
print(JM1_original.shape)

(10880, 22)


# **1) 이상치 데이터 제거 후 모델 학습/예측/평가**

- 21개의 feature 들 중에서 가장 상관계수가 높은 'loc'에 대해서만 이상치를 찾아서 제거함.

- IQR을 이용해 이상치를 검출하는 함수를 생성한 뒤, 이를 이용해 검출된 이상치를 삭제한다.


In [54]:
def get_outlier(df=None, column=None, weight=1.5):
  #fraud에 해당하는 column 데이터만 추출, 1/4 분위와 3/4 분위 지점을 np.percentile로 구함.
  fraud = df[df['defects']==0][column]
  quantile_25 = np.percentile(fraud.values,25)
  quantile_75 = np.percentile(fraud.values,75)

  #IQR을 구하고 IQR에 1.5를 곱해 최댓값과 최솟값 지점을 구함
  iqr = quantile_75 - quantile_25
  iqr_weight = iqr * weight
  lowest_val = quantile_25 - iqr_weight
  highest_val = quantile_75 + iqr_weight

  #최댓값보다 크거나, 최솟값보다 작은 값을 이상치 데이터로 설정하고 DataFrame index 반환
  outlier_index = fraud[(fraud<lowest_val)|(fraud>highest_val)].index
  return outlier_index

In [55]:
outlier_index = get_outlier(df=JM1_original, column='loc', weight=1.5)
print("이상치 데이터 인덱스:", outlier_index)

이상치 데이터 인덱스: Int64Index([ 2107,  2109,  2110,  2111,  2113,  2114,  2115,  2116,  2121,
             2127,
            ...
            10840, 10841, 10842, 10843, 10855, 10856, 10858, 10859, 10860,
            10862],
           dtype='int64', length=779)


In [56]:
# 이상치 제거
JM1_original.drop(outlier_index, axis=0, inplace=True)
print(JM1_original.shape)

(10101, 22)


In [57]:
# 입력 데이터
input_data = JM1_original.iloc[:,:21]
print("input_data shape : ",input_data.shape)

input_data shape :  (10101, 21)


In [58]:
input_data = np.asarray(input_data,dtype=np.float)

In [59]:
input_data[0]

array([1.1, 1.4, 1.4, 1.4, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 2. ,
       2. , 2. , 2. , 1.2, 1.2, 1.2, 1.2, 1.4])

In [60]:
input_data.shape

(10101, 21)

In [61]:
# 정답 레이블
y_label = JM1_original.iloc[:,21].to_numpy()
print("y_label shape : ",y_label.shape)

y_label shape :  (10101,)


In [62]:
y_label[0]

False

In [63]:
cnt_true =0
cnt_false=0
# 정답 레이블 확인 해보기
for i in range(len(y_label)):
  if y_label[i]:
    cnt_true+=1
  else:
    cnt_false+=1

print("fault-prone: ", cnt_true)
print("non-fault-prone: ", cnt_false)

fault-prone:  2103
non-fault-prone:  7998


In [64]:
# 레이블 벡터화 
y_label = np.asarray(y_label).astype('float32')

In [65]:
y_label.shape

(10101,)

In [66]:
from sklearn import preprocessing

# 정규화 0-1로 하기
data_normalizer = preprocessing.MinMaxScaler() # 데이터를 0~1 범위로 점철되게 하는 함수 call
data_normalized = data_normalizer.fit_transform(input_data) # 데이터를 0~1 범위로 점철되게 함수 수행
data_normalized.shape

(10101, 21)

In [67]:
data_normalized[0]

array([2.90613194e-05, 8.52878465e-04, 2.87769784e-03, 9.97506234e-04,
       1.54010188e-04, 1.60805353e-05, 1.00000000e+00, 3.18058376e-03,
       2.28158237e-03, 4.18278349e-08, 4.82374768e-02, 7.52901028e-07,
       7.08215297e-04, 5.81395349e-03, 4.47427293e-03, 1.85185185e-02,
       2.91970803e-03, 1.16959064e-03, 2.21402214e-04, 3.97219464e-04,
       4.84848485e-04])

# **10-fold cross-valid**

In [68]:
from sklearn.model_selection import StratifiedKFold
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping

# 5개의 파일로 쪼갬
n_fold = 5
seed = 0
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

#빈 metrics 배열
accuracy = []
loss = []
auc = []
FPR = []
FNR = []

In [69]:
from typing import Pattern
# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='loss', patience=100)

In [ ]:
for train, test in skf.split(data_normalized, y_label):
  model = models.Sequential()
  model.add(layers.Dense(64, input_shape = (21, )))
  model.add(layers.BatchNormalization())
  model.add(layers.Activation('relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(64))
  model.add(layers.BatchNormalization())
  model.add(layers.Activation('relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(64))
  model.add(layers.BatchNormalization())
  model.add(layers.Activation('relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(64))
  model.add(layers.BatchNormalization())
  model.add(layers.Activation('relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(64))
  model.add(layers.BatchNormalization())
  model.add(layers.Activation('relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1))
  model.add(layers.Activation('sigmoid'))

  model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=[tf.keras.metrics.TruePositives(),tf.keras.metrics.FalseNegatives(),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TrueNegatives(),"AUC"])
  hist = model.fit(data_normalized[train], y_label[train], epochs=600, batch_size=5, verbose=1, callbacks=[early_stopping_callback])
  TP = model.evaluate(data_normalized[test],y_label[test])[1]
  FN = model.evaluate(data_normalized[test],y_label[test])[2]
  FP = model.evaluate(data_normalized[test],y_label[test])[3]
  TN = model.evaluate(data_normalized[test],y_label[test])[4]
  k_accuracy = "%.4f"%((TP+TN)/(TP+FP+FN+TN))
  k_loss = "%.4f"%(model.evaluate(data_normalized[test],y_label[test])[0])
  k_FPR = "%.4f"%((FP)/(FP+TN))
  k_FNR="%.4f"%((FN)/(TP+FN))
  k_auc = "%.4f"%(model.evaluate(data_normalized[test],y_label[test])[5])
  accuracy.append(k_accuracy)
  loss.append(k_loss)
  FNR.append(k_FNR)
  FPR.append(k_FPR)
  auc.append(k_auc)

In [ ]:
print("\n %.f fold accuracy:" % n_fold, accuracy)
print("모든 fold의 평균 accuracy: ", (float(accuracy[0]) + float(accuracy[1]) + float(accuracy[2]) + float(accuracy[3]) + float(accuracy[4])) / 5)

print("\n %.f fold loss:" % n_fold, loss)
print("모든 fold의 평균 loss: ", (float(loss[0]) + float(loss[1]) + float(loss[2]) + float(loss[3]) + float(loss[4])) / 5)

print("\n %.f fold FNR:" % n_fold, FNR)
print("모든 fold의 평균 FNR: ", (float(FNR[0]) + float(FNR[1]) + float(FNR[2]) + float(FNR[3]) + float(FNR[4])) / 5)

print("\n %.f fold FPR:" % n_fold, FPR)
print("모든 fold의 평균 FPR: ", (float(FPR[0]) + float(FPR[1]) + float(FPR[2]) + float(FPR[3]) + float(FPR[4])) / 5)

print("\n %.f fold auc:" % n_fold, auc)
print("모든 fold의 평균 auc: ", (float(auc[0]) + float(auc[1]) + float(auc[2]) + float(auc[3]) + float(auc[4])) / 5)